In [1]:
import numpy as np
import pandas as pd
import pydst
Dst = pydst.Dst(lang='en')


In [2]:
account = Dst.get_data(table_id = "REGK11", variables = {"OMRÅDE": ["*"], "FUNK1": ["X"], "DRANST": ["1"], "ART": ["TOT"], "PRISENHED": ["INDL"], "Tid": ["*"]})
budget = Dst.get_data(table_id = "BUDK1", variables = {"REGI07A": ["*"], "FUNK1": ["X"], "DRANST": ["1"], "ART": ["TOT"], "PRISENHED": ["INDL"], "Tid": ["*"]})
people = Dst.get_data(table_id = "INDAMP01", variables = {"OMRÅDE": ["*"], "KØN": ["TOT"], "ALDER": ["TOT"], "PERSG": ["IALT"], "Tid": ["*"], "BNØGLE": ["PER"]})


account har årene 2011-2018

budget har årene 2009-2019

people har årene 2006-2017

In [3]:
account.drop(["FUNK1", "DRANST", "ART", "PRISENHED"], axis=1, inplace=True)
budget.drop(["FUNK1", "DRANST", "ART", "PRISENHED"], axis=1, inplace=True)
people.drop(["KØN", "ALDER", "PERSG", "BNØGLE"], axis=1, inplace=True)


In [4]:
budget.rename(columns = {"REGI07A":"Municipality", "TID": "Year", "INDHOLD":"Budget"}, inplace = True)
account.rename(columns = {"OMRÅDE":"Municipality", "TID": "Year", "INDHOLD":"account"}, inplace = True)
people.rename(columns = {"OMRÅDE":"Municipality", "TID": "Year", "INDHOLD":"people"}, inplace = True)

print(people.head())
print(budget.head())
print(account.head())
account.info()
people.info()
budget.info()

  Municipality  Year  people
0       Holbæk  2011   69521
1         Faxe  2011   35277
2   Kalundborg  2011   49088
3     Ringsted  2011   32890
4     Slagelse  2011   77442
  Municipality  Year  Budget
0   Jammerbugt  2015   64367
1       Assens  2015   63837
2    Norddjurs  2015   65445
3        Greve  2015   57967
4      Herning  2015   59441
  Municipality  Year  account
0     Hillerød  2012    62180
1       Solrød  2012    53166
2       Dragør  2012    53794
3     Gribskov  2012    61529
4     Gentofte  2012    54695
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832 entries, 0 to 831
Data columns (total 3 columns):
Municipality    832 non-null object
Year            832 non-null int64
account         832 non-null int64
dtypes: int64(2), object(1)
memory usage: 19.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1380 entries, 0 to 1379
Data columns (total 3 columns):
Municipality    1380 non-null object
Year            1380 non-null int64
people          1380 non-null in

In [9]:
for val in ["Region", "Province", "All Denmark"]:
    for count, dataset in enumerate([people, budget, account]):
        rowstodrop = dataset.loc[lambda df: df["Municipality"].str.contains(val)].index
        dataset.drop(rowstodrop, axis=0, inplace=True)

def an():
    """dafd"""


           Municipality  Year  people
0                Holbæk  2011   69521
1                  Faxe  2011   35277
2            Kalundborg  2011   49088
3              Ringsted  2011   32890
4              Slagelse  2011   77442
5                Stevns  2011   21855
6                  Sorø  2011   29499
7                 Lejre  2011   26764
8               Lolland  2011   46111
9               Næstved  2011   80963
10         Guldborgsund  2011   62583
11          Vordingborg  2011   46170
12             Bornholm  2011   41896
13           Middelfart  2011   37701
14               Assens  2011   41635
15      Faaborg-Midtfyn  2011   51926
16           Kerteminde  2011   23758
17               Nyborg  2011   31499
18               Odense  2011  190245
19            Svendborg  2011   58713
20             Nordfyns  2011   29507
21            Langeland  2011   13322
22                  Ærø  2011    6663
23            Haderslev  2011   56117
24              Billund  2011   26155
25          

In [6]:
a = []
for i in range(2011,2018):
    a.append(str(i))

budget = budget.loc[budget["Year"].isin(a)]
people = people.loc[people["Year"].isin(a)]
account = account.loc[account["Year"].isin(a)]

In [7]:
account.info()
people.info()
budget.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 686 entries, 0 to 831
Data columns (total 3 columns):
Municipality    686 non-null object
Year            686 non-null int64
account         686 non-null int64
dtypes: int64(2), object(1)
memory usage: 21.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 686 entries, 0 to 1378
Data columns (total 3 columns):
Municipality    686 non-null object
Year            686 non-null int64
people          686 non-null int64
dtypes: int64(2), object(1)
memory usage: 21.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 686 entries, 0 to 888
Data columns (total 3 columns):
Municipality    686 non-null object
Year            686 non-null int64
Budget          686 non-null int64
dtypes: int64(2), object(1)
memory usage: 21.4+ KB


In [13]:
print(people[people["Municipality"].str.contains("Region")]==True)

Empty DataFrame
Columns: [Municipality, Year, people]
Index: []


In [8]:
list(set(list(people["Municipality"]))-set(list(budget["Municipality"])))

[]